In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import os
import csv
import re
import traceback
import datetime
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
def human_login(elem, login_info):
    info_len = len(login_info)
    if info_len < 1:
        print("Invalid LoginInfo")
        return
    i = 0
    while i < info_len:
        choice = random.randint(1, 5)
        if choice == 5 and random.randint(0, 5) > 3:
            if i == 0:
                continue
            else:
                elem.send_keys(Keys.BACKSPACE)
                i -= 1
        else:
            elem.send_keys(login_info[i])
            i += 1
            time.sleep(0.1 + choice * 0.02)




In [27]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})
# 进入淘宝官网
driver.get('https://www.taobao.com/')
driver.maximize_window()
# 定位到登录键
elem = driver.find_element("class name", "h")
elem.click()
time.sleep(2)
# 定位元素到账号一栏
elem = driver.find_element("id", "fm-login-id")
# 开始“类人”输入账号
human_login(elem, "18089439152")
# 睡一会
time.sleep(0.5)
# 定位元素到密码一栏
elem = driver.find_element("id", "fm-login-password")
# 开始“类人”输入密码
human_login(elem, "7895123tb")
# 按下 enter 键
elem.send_keys(Keys.RETURN)
# 再睡会
# 这里睡的时间那么长，是为了防止出现验证界面（系统自己检测成功后会自动跳转）
time.sleep(3)
# 这里的定位很简单

In [5]:
def save_csv(data_dic,file_name,i):
    file_exists = os.path.exists(file_name)

    with open(file_name, 'a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data_dic.keys())

        # 如果文件不存在，写入表头
        if not file_exists:
            writer.writeheader()

        # 写入数据
        writer.writerow(data_dic)
    print(i,'turns:''success')

In [6]:
def scroll(driver): 
    total_height = driver.execute_script("return document.body.scrollHeight")

    # 计算位置
    scroll_to = total_height * 0.20

    # 每次滚动增加的高度
    increment = 200  # 您可以根据需要调整这个值

    # 当前滚动位置
    current_scroll_position = 0

    while current_scroll_position < scroll_to:
        # 增加滚动位置
        current_scroll_position += increment

        # 滚动到新位置
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

        # 短暂延时
        time.sleep(0.1)  # 每次滚动后的等待时间，可以根据需要调整

        # 防止滚动超出目标位置
        if current_scroll_position > scroll_to:
            driver.execute_script(f"window.scrollTo(0, {scroll_to});")
            break
    time.sleep(2)

In [3]:
df=pd.read_csv('cleanedData.csv')
urls=df['product_link']

In [28]:
for i in complete:
    driver.get(urls[i])
    scroll(driver)
    try:
        time.sleep(5)
        '''
        //*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[1]/div[2]
        //*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[2]/div[2]
        '''
        #店铺评分

        product_discrp=driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[1]/div[2]').text
        mkt_service=driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[2]/div[2]').text
        mailing=driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/a/div[3]/div[3]/div[2]').text

        '''
        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[1]/div/ul/li[1]/img
        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[1]/div/ul/li[2]/img
        '''

        #商品链接

        gallery=driver.find_element(By.XPATH,'//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[1]/div/ul')
        li_elements = gallery.find_elements(By.TAG_NAME, 'li')

        src_list = []

        # 遍历每个 'li' 元素，找到其中的 'img' 标签并获取 'src' 属性
        for li in li_elements:
            img_element = li.find_element(By.TAG_NAME, 'img')
            src = img_element.get_attribute('src')
            src_list.append(src)


        #商品详情
        '''
        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]

        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]/div[1]
        //*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]/div[2]
        #<div class="Attrs--attrSection--2_G8xGa"><span class="Attrs--attr--33ShB6X" title="品牌：顶峰保罗">品牌：顶峰保罗</span><span class="Attrs--attr--33ShB6X" title="尺码：S M L XL 2XL 3XL">尺码：S M L XL 2XL 3XL</span><span class="Attrs--attr--33ShB6X" title="面料分类：其他">面料分类：其他</span></div>
        '''
        product_details = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[2]/div[3]/div[3]/div[1]/div[4]')

        # 获取 div 元素的 HTML 内容
        div_html = product_details.get_attribute('innerHTML')

        # 使用 BeautifulSoup 解析 HTML
        soup = BeautifulSoup(div_html, 'html.parser')

        # 初始化一个字典来存储商品信息
        product_detail = {}

        # 遍历所有的 span 元素
        for span in soup.find_all('span', class_='Attrs--attr--33ShB6X'):
            # 提取每个 span 的 title 属性
            title_text = span.get('title')
            if title_text:
                # 分割 title 文本来获取键和值
                key, value = re.split(r'：', title_text, maxsplit=1)
                product_detail[key] = value
        pro_info={"SN":i,"product_discrip":product_discrp,'mkt_service':mkt_service,'mailing':mailing,'src_list':src_list,'product_detail':product_detail}

        save_csv(pro_info,'product_info2.csv',i)
        
        
        ##########################################
    except NoSuchElementException:
        try:
            time.sleep(5)
            '''           
            #描述
            //*[@id="header-content"]/div[2]/div[2]/ul/li[1]/span[2]/em
            #服务
            //*[@id="header-content"]/div[2]/div[2]/ul/li[2]/span[2]/em
            #物流
            //*[@id="header-content"]/div[2]/div[2]/ul/li[3]/span[2]/em
            '''
            #店铺评分
            shops_scores=driver.find_element(By.XPATH, '//*[@id="header-content"]/div[2]/div[2]/ul')
            shop_li_elements = shops_scores.find_elements(By.TAG_NAME, 'li')

            product_discrp = shop_li_elements[0].find_element(By.XPATH, './span[2]/em').text

            mkt_service = shop_li_elements[1].find_element(By.XPATH, './span[2]/em').text

            mailing = shop_li_elements[2].find_element(By.XPATH, './span[2]/em').text



            #图片
            '''
            #<img data-src="//gd1.alicdn.com/imgextra/i3/0/O1CN0111PCrm20ToKnPvRWQ_!!0-item_pic.jpg_50x50.jpg" src="//gd1.alicdn.com/imgextra/i3/0/O1CN0111PCrm20ToKnPvRWQ_!!0-item_pic.jpg_50x50.jpg_.webp">
            //*[@id="J_UlThumb"]/li[1]/div/a/img
            //*[@id="J_UlThumb"]/li[2]/div/a/img
            '''

            gallery=driver.find_element(By.XPATH,'//*[@id="J_UlThumb"]')
            li_elements = gallery.find_elements(By.TAG_NAME, 'li')

            src_list = []

            # 遍历每个 'li' 元素，找到其中的 'img' 标签并获取 'src' 属性
            for li in li_elements:
                img_element = li.find_element(By.TAG_NAME, 'img')
                src = img_element.get_attribute('src')
                src_list.append(src)

            #商品详情
            '''
            #<li title="棉致">品牌:&nbsp;棉致</li>
            //*[@id="attributes"]/ul
            '''
            ul_element = driver.find_element(By.XPATH, '//*[@id="attributes"]/ul')

            # 初始化一个字典来存储产品详情
            product_detail = {}

            # 遍历 'ul' 元素下的所有 'li' 元素
            for li in ul_element.find_elements(By.TAG_NAME, 'li'):
                # 获取 'li' 元素的 'title' 属性
                key = li.get_attribute('title')

                # 获取 'li' 元素的文本，并去除冒号前的文本（如“品牌:&nbsp;”）
                text = li.text
                value = re.split(r':\s*&nbsp;', text, maxsplit=1)[1] if ':&nbsp;' in text else text

                product_detail[key] = value


            pro_info={"SN":i,"product_discrip":product_discrp,'mkt_service':mkt_service,'mailing':mailing,'src_list':src_list,'product_detail':product_detail}

            save_csv(pro_info,'product_info2.csv',i)


            
     
        except Exception as e:
            # 获取完整的错误信息

            error_info = traceback.format_exc()

            # 获取当前时间
            current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # 将错误信息、当前时间和 URL 写入文件
            with open('errors2.csv', 'a') as file:
                file.write(f"{i},{current_time},{urls[i]},{error_info.replace(',', ';')}\n")
            print(i,' turns:failed')
            time.sleep(5)

 
        

1152 turns:success
1153  turns:failed
1156  turns:failed
1157  turns:failed
1159 turns:success
1167 turns:success
1171  turns:failed
1174  turns:failed
1175 turns:success
1176 turns:success
1178 turns:success
1179  turns:failed
1180  turns:failed
1182  turns:failed
1184 turns:success
1185 turns:success
1186  turns:failed
1187 turns:success
1188 turns:success
1189 turns:success
1190  turns:failed
1191 turns:success
1192 turns:success
1193 turns:success
1194 turns:success
1195 turns:success
1196  turns:failed
1197 turns:success
1198 turns:success
1199 turns:success
1200 turns:success
1201 turns:success
1202 turns:success
1203 turns:success
1204 turns:success
1205 turns:success
1206 turns:success
1207  turns:failed
1208 turns:success
1209 turns:success
1210 turns:success
1211 turns:success
1212 turns:success
1213 turns:success
1214  turns:failed
1215 turns:success
1216 turns:success
1217 turns:success
1218 turns:success
1219 turns:success
1220 turns:success
1221 turns:success
1222 turns:s

1602 turns:success
1603 turns:success
1604 turns:success
1605 turns:success
1606 turns:success
1607 turns:success
1608  turns:failed
1609 turns:success
1610 turns:success
1611 turns:success
1612 turns:success
1613  turns:failed
1614 turns:success
1615 turns:success
1616 turns:success
1617 turns:success
1618 turns:success
1619 turns:success
1620 turns:success
1621 turns:success
1622 turns:success
1623 turns:success
1624 turns:success
1625 turns:success
1626 turns:success
1627  turns:failed
1628  turns:failed
1629  turns:failed
1630 turns:success
1631 turns:success
1632 turns:success
1633 turns:success
1634  turns:failed
1635 turns:success
1636 turns:success
1637 turns:success
1638 turns:success
1639 turns:success
1640 turns:success
1641 turns:success
1642  turns:failed
1643 turns:success
1644 turns:success
1645 turns:success
1646 turns:success
1647 turns:success
1648 turns:success
1649 turns:success
1650 turns:success
1651 turns:success
1652 turns:success
1653  turns:failed
1654 turns:s

In [25]:
finished=pd.read_csv('product_info2.csv')['SN'].to_list()
complete=[i for i in list(range(1000,2000)) if i not in finished and i>1150]


In [19]:
driver.get(urls[1000])

In [ ]:
s=0
for i in range(1,16+1):
    for j in range(1,18-i):
        for k in range(1,19-i-j):
            s+=comb(i,18)*comb(j,18-i)*comb(k,18-i-j)